# (1) Install dependencies

In [ ]:
!apt update
!apt-get install libdw-dev

#(1) Download the BMF Release library and extract it to the local environment

In [ ]:
!pip install BabitMF

#(3) Configure the environment variable to reference the BMF C library

In [ ]:
!bmf_env

In [ ]:
%env C_INCLUDE_PATH=/usr/local/lib/python3.10/dist-packages/bmf/include
%env CPLUS_INCLUDE_PATH=/usr/local/lib/python3.10/dist-packages/bmf/include
%env LIBRARY_PATH=/usr/local/cuda/lib64/stubs:/usr/local/lib/python3.10/dist-packages/bmf/lib
%env LD_LIBRARY_PATH=/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/lib/python3.10/dist-packages/bmf/lib

In [ ]:
!echo $LD_LIBRARY_PATH

In [ ]:
!wget https://github.com/BabitMF/bmf/releases/download/files/files.tar.gz
!tar -zvxf files.tar.gz

In [ ]:
%%writefile /content/sync_mode.cpp
#include "builder.hpp"
#include "nlohmann/json.hpp"


int main()
{
    std::string output_file = "./sync_mode.mp4";

    bmf::builder::Graph graph = bmf::builder::Graph(bmf::builder::NormalMode);

    // create sync modules
    nlohmann::json decoder_option = {
        {"input_path", "./files/big_bunny_10s_30fps.mp4"}
    };
    auto decoder = graph.Sync(std::vector<int> {}, std::vector<int> {0,1}, decoder_option, "c_ffmpeg_decoder");

    nlohmann::json scale_option = {
        {"name", "scale"},
        {"para", "320:250"}
    };
    auto scale = graph.Sync(std::vector<int> {0}, std::vector<int> {0},
        bmf_sdk::JsonParam(scale_option), "c_ffmpeg_filter");

    nlohmann::json volume_option = {
        {"name", "volume"},
        {"para", "volume=3"}
    };
    auto volume = graph.Sync(std::vector<int> {0}, std::vector<int> {0}, volume_option, "c_ffmpeg_filter");

    nlohmann::json encoder_option = {
        {"output_path", output_file}
    };
    auto encoder = graph.Sync(std::vector<int> {0,1}, std::vector<int> {}, encoder_option, "c_ffmpeg_encoder");

    // call init if necessary, otherwise we skip this step
    graph.Init(decoder);
    graph.Init(scale);
    graph.Init(volume);
    graph.Init(encoder);

    // process video/audio by sync mode
    while (1) {
        auto decoded_frames = graph.Process(decoder, bmf::builder::SyncPackets());
        bool has_next = false;
        for (const auto &stream : decoded_frames.packets) {
            if (!stream.second.empty()) {
                has_next = true;
                if (stream.first == 0) {
                    bmf::builder::SyncPackets input_scale;
                    input_scale.Insert(0, decoded_frames[0]);
                    auto scaled_frames = graph.Process(scale, input_scale);

                    bmf::builder::SyncPackets input_encoder;
                    input_encoder.Insert(0, scaled_frames[0]);
                    graph.Process(encoder, input_encoder);
                    //encoder.ProcessPkts(input_encoder);
                } else if (stream.first == 1) {
                    bmf::builder::SyncPackets input_volume;
                    input_volume.Insert(0, decoded_frames[1]);
                    auto volume_frames = graph.Process(volume, input_volume);
                    //auto volume_frames = volume.ProcessPkts(input_volume);

                    bmf::builder::SyncPackets input_encoder;
                    input_encoder.Insert(1, volume_frames[0]);
                    graph.Process(encoder, input_encoder);
                    //encoder.ProcessPkts(input_encoder);
                }
            }
        }
        if (!has_next) {
            break;
        }
    }

    // call close if necessary, otherwise we skip this step
    graph.Close(decoder);
    graph.Close(scale);
    graph.Close(volume);
    graph.Close(encoder);

}


In [ ]:
%%writefile /content/CMakeLists.txt
file(GLOB SRCS *.cpp *.h)
add_library(nlohmann INTERFACE IMPORTED GLOBAL)
# You may need this option on the version of BabitMF below 0.0.9, because the base image BMF uses for release is updated from Centos7 to AlmaLinux8
#add_definitions(-D_GLIBCXX_USE_CXX11_ABI=0)
add_executable(sync_mode ${SRCS})

target_link_libraries(
    sync_mode
    PRIVATE
    bmf_module_sdk hmp engine nlohmann
)

In [ ]:
!rm -rf CMakeCache.txt CMakeFiles cmake_install.cmake Makefile

In [ ]:
!cmake .
!make

In [ ]:
!./sync_mode

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 800):
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return f"""
  <video width={video_width} controls>
    <source src="{video_url}">
  </video>
  """

video_url1 = show_video('./files/big_bunny_10s_30fps.mp4')
video_url2 = show_video('sync_mode.mp4')

html = video_url1 + video_url2
HTML(html)